# Candidate Line Selection (Dropped)

- Introduction
- Import: packages, utils, data, etc.
- Transmission opportunity metric: $\tau^{ac/dc}_{ij}$
- Candidate selection LP model: $\hat{f^{ac/dc}_{ij}}, \hat{\lambda_i}, \hat{\lambda_j}$
- Candidate selection filter: $\forall ij\in C^{ac/dc} \rightarrow \tau^{ac/dc}_{ij} < 1 \rightarrow ij=1,...,|C^{ac/dc}_{reduced}|$

## Introduction

Traditionally, the considered candidate lines for transmission expansion studies are proposed by power system operators and planners based on their subject matter expertise. This is because there are too many possible combination of nodes, and essentially most of these combinations are bad candidates. Formally, given $n$ number of nodes, the possible number of candidate lines, $|C|$,  can be described by the following equation.

$\begin{align}
 |C| = \frac{n!}{2(n-2)!}
 \end{align}$

For small $n$, it is still possible to consider all candidate lines. For example, $n=10 \rightarrow |C|=45$. However, for larger $n$, i.e. 100,1000 and 10000, the number of possible candidate lines significanltly increases to 4950, 499500 and 49995000 respectively. Planning studies usually model their problem as mixed-integer program which is very difficult to solve compared to linear programs. Having these many variables will result in very long computational time for solving mathematical programs, which might not be acceptable for certain planning studies.

However, directly resorting to predefined set of candidate lines might not be the best idea. It is possible for system operators to overlook some potential candidates, and this might lead to suboptimal findings. This is especially true for cross-border planning studies, because the criteria of a good candidate within a system might not be the same as the criteria for a good candidate when considering multiple systems.

The aim of this section is to provide an analytical approach in finding a set of candidates with the highest potential, based on the concept of congestion rent. This will be used for both HVAC and HVDC lines selection.

## Transmission recovery metric

Having discussed the method to calculate the length of potential candidates and approximate their respective parameters, we now propose a strategy for candidate lines selection based on the concept of congestion rent. We define a metric called transmission recovery for each candidate line $ij$ as the ratio of expected congestion rent between node $i$ and $j$ and investment cost of $ij$. Formally, we define transmission recovery of AC line and HVDC link $ij$, denoted by $\tau^{ac}_{ij}$ and $\tau^{dc}_{ij}$ respectively, as follows. 

$\begin{align}
\tau^{ac}_{ij} = \frac{|Y|\times \omega_p f^{ac}_{ij,p}|\lambda^{en}_{i,p} - \lambda^{en}_{j,p}|}{Cost^{ac-f} + |Y|\times Cost^{ac-v}Length_{ij}cap^{ac}_{ij}} \\
\tau^{dc}_{ij} = \frac{|Y|\times f^{dc}_{ij}|\lambda^{en}_{i,p} - \lambda^{en}_{j,p}|}{Cost^{dc-f} + |Y|\times Cost^{dc-v}Length_{ij}cap^{dc}_{ij}} 
\end{align}$

where $Y$ is the expected number of years to recover transmission investment, $\rho$ is the expected yearly duration percentage for the system to operate in peak demand condition, and $f^{ac}_{ij}, f^{dc}_{ij}$ are the power flows in AC line and HVDC link $ij$ respectively. $\lambda_i$ and $\lambda_j$ are the locational marginal prices of energy at node $i$ and $j$ respectively. Note that ancillary services marginal prices are not explicitly considered in the transmission recovery metric, however the cost of ancillary services will still be considered in the objective function of the candidate selection optimisation model that we will discuss shortly to obtain $f^{ac}_{ij}, f^{ac}_{ij},\lambda_i,\lambda_j,newcap_{ij}$ and $cczc_{ij}$.

### Candidate Line LP Model

We consider the network at the end of the planning horizon operating at peak demand condition, i.e. if we are planning for 15 years, the expected peak demand and generation of the 15th year are considered. Thus, the following candidate selection linear program (LP) model, which is essentially the linear program equivalent of transmission expansion can be formulated as follows.

$\begin{align}
&\min \quad 8760\rho\sum_{g\in G} Cost^{op}_g + \sum_{ij\in C^{ac}}Cost^{ac-var}Dist_{ij}ccap_{ij} + \sum_{ij\in C^{dc}}Cost^{dc-var}Dist_{ij}cczc_{ij}\\
&\text{subject to}\notag\\
&\notag\\
&\sum_{g\in G_i} p_g + \sum_{ij\in C^{ac}} (f^{ac}_{ji} - f^{ac}_{ij}) + \sum_{ij\in C^{dc}} (f^{dc}_{ji} - f^{dc}_{ij}) - Load_i = 0 \quad \forall i\in N\\
&f^{ac} - \frac{1}{x_{ij}}(\delta_i - \delta_j) = 0\quad \forall ij\in C^{ac}\\
&|f^{ac}_{ij}| - (Ecap_{ij} + ccap_{ij}) \leq 0 \quad \forall ij\in C^{ac}\\
&|f^{dc}_{ij}| - (Eczc_{ij} + cczc_{ij}) \leq 0 \quad \forall ij\in C^{dc}\\
&p_g, ru_g, rd_g \geq 0 \quad \forall g\in G\\
&p_g + ru_g - P^{max}_g \leq 0 \quad \forall g\in G\\
&p_g - rd_g - P^{min}_g \geq 0 \quad \forall g\in G\\
&\frac{ru_g}{Ramp^{reg-up}_g} - 10 \leq 0 \quad \forall g\in G\\
&\frac{rd_g}{Ramp^{reg-down}_g} - 10 \leq 0 \quad \forall g\in G\\
\end{align}$

where $Cost^{op}_g$ is defined and the remaining variables are already discussed in the background. The objective function of this model is to minimise the operating cost and the variable transmission investment cost. The reason this model is not used for the actual transmission planning is because it removes the lumpiness nature of the problem itself, i.e. it is very unlikely to build a new line with 1MW capacity. This happens because the model does not consider the binary variables $\alpha$ and $\beta$, consequently the high fixed cost associated with the investment decisions. Nonetheless, there is still value in formulating this LP. Due to the maturity of LP solvers, we can now consider the large number of variables associated with all possible candidate lines, which is not the case for mixed-integer solvers (yet). Thus, this allows us to identify potential candidate AC lines and HVDC links to be considered in the actual TEP model.

$\begin{align}
f = &\min \sum_{a\in A}\Bigg[\sum_{p\in P}Y \times Hours_p\bigg(\sum_{g\in G_a}Cost^{en}_g en_{g,p} + \sum_{g\in G_a}(Cost^{ru}_g ru_{g,p} + Cost^{rd}_g rd_{g,p}) \\
&\qquad+ \sum_{v\in V} Cost^{vre}_v vre_{v,p} + \sum_{i\in N_a} Cost^{voll} ens_{i,p}\bigg)\\
& \qquad + \sum_{ij\in C^{ac}_a}Cost^{ac-v}Length_{ij}cap^{ac}_{ij} + \sum_{ij\in C^{dc}_a}Cost^{dc-v}Length_{ij}cap^{dc}_{ij}\Bigg] \\
&\qquad + \sum_{ij\in C^{dc}_{cross}}Cost^{dc-v}Length_{ij}cap^{dc}_{ij}\\
&\text{subject to} \notag\\
&\sum_{g\in G_i} en_{g,p} +\sum_{v\in V_i} vre_{v,p} + \sum_{j\in C^{ac}_i}(f^{ac}_{ji,p} - f^{ac}_{ij,p}) + \sum_{j\in C^{dc}_i}(f^{dc}_{ji,p} - f^{dc}_{ij,p}) = Load_{i,p}\quad \forall i\in N, p\in P\\
&|f^{ac}_{ij,p}| \leq Excap^{ac}_{ij} + cap^{ac}_{ij} \quad \forall ij\in C^{ac}, p\in P\\
&|f^{dc}_{ij,p}| \leq Excap^{dc}_{ij} + cap^{dc}_{ij} \quad \forall ij\in C^{dc}, p\in P\\
&en_{g,p} + ru_{g,p} \leq P^{max}_{g} \quad \forall g\in G, p\in P\\
&en_{g,p} - rd_{g,p} \geq P^{min}_{g} \quad \forall g\in G, p\in P\\
&vre_{v,p} \leq CF_{v,p}P^{max}_{v} \quad \forall v\in V, p\in P\\
&\sum_{g\in G_a} ru_{g,p} \geq Req^{ru}_a \quad \forall a\in A\\
&\sum_{g\in G_a} rd_{g,p} \geq Req^{rd}_a \quad \forall a\in A\\
&en_{g,p}, ru_{g,p}, rd_{g,p}, vre_{v,p} \geq 0 \quad \forall g\in G, v\in V, p\in P
\end{align}$